# Augmentation

Due to the fact that the data is inbalanced across categories I was searching for strategies to fix that. One abbroach I found interesting was to translate the news text of underrepresented categories into a different language (english) and back. This leads to a slightly modified version of the text with the same label.

To do that i was searching for a translation model on hugging face. I tested several on the model exploration page and took the one that provided 

In [27]:
!pip install torch

In [ ]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
from pandas import concat, read_parquet, DataFrame, set_option
from data import file
from tqdm import tqdm

In [ ]:
# apply progress bar on pandas operations
tqdm.pandas()
set_option('display.max_colwidth', None)

In [ ]:
mname_de_en = "facebook/wmt19-de-en"
tokenizer_de_en = FSMTTokenizer.from_pretrained(mname_de_en)
model_de_en = FSMTForConditionalGeneration.from_pretrained(mname_de_en)

def translate_de_en(input):
    input_ids = tokenizer_de_en.encode(input, return_tensors="pt")
    decoded = tokenizer_de_en.decode(model_de_en.generate(input_ids)[0], skip_special_tokens=True)
    return decoded

mname_en_de = "facebook/wmt19-en-de"
tokenizer_en_de = FSMTTokenizer.from_pretrained(mname_en_de)
model_en_de = FSMTForConditionalGeneration.from_pretrained(mname_en_de)


def translate_en_de(input):
    input_ids = tokenizer_en_de.encode(input, return_tensors="pt")
    outputs = model_en_de.generate(input_ids)
    decoded = tokenizer_en_de.decode(outputs[0], skip_special_tokens=True)
    return decoded

def augment(input):
    return translate_en_de(translate_de_en(input))

In [9]:
data = read_parquet(file.news_articles_raw)


In [6]:
config = {
    'Etat': 10,
    'Inland': 10,
    'International': 3,
    'Kultur': 5,
    'Panorama': 0,
    'Sport': 5,
    'Web': 0,
    'Wirtschaft': 0,
    'Wissenschaft': 3,
}

def fake_tranlsate(input):
    return f"augmented: {input}"

list = []
for (category, count) in config.items():
    selection = data.loc[data.label==category][0:count]
    text_augmented = selection.text_original.progress_map(augment)
    list.append(DataFrame(concat([text_augmented, selection.label], axis=1)))
    

100%|██████████| 5/5 [03:49<00:00, 45.86s/it]
0it [00:00, ?it/s]
100%|██████████| 5/5 [02:18<00:00, 27.78s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 3/3 [01:47<00:00, 35.70s/it]


In [17]:
augmented_only = DataFrame(concat(list))
augmented_only.to_parquet(path=file.news_articles_augmented_only)

In [18]:
augmented = concat([data, augmented_only])
augmented.to_parquet(path=file.news_articles_augmented)

In [16]:
augmented_only

,text_original,label
7,"Wien - Nach der Wahl hat man die Wahl: Während im Tal die Regierungsparteien um Geschlossenheit kämpfen, ist auf dem Küniglberg derzeit alles eher gespalten: Am Montagnachmittag trifft sich der ORF-Stiftungsrat zu einer außerordentlichen Arbeitssitzung, auf der Sitzungsliste standen nach Angaben des Senats Themen mit Konfliktpotenzial: Punkt eins: der Fortschritt der Sanierungsarbeiten. Wie berichtet, informierte Generaldirektor Alexander Wrabetz den Aufsichtsrat im Dezember über die Überschreitung der Baukosten in Millionenhöhe. Am Montag erhielt der Stiftungsrat einen Sachstandsbericht über den Medienstandort über Zeitplan und Kosten und besichtigte auch die Baustelle. Thomas Zach, Vorsitzender des Finanzausschusses und Chef des ÖVP-Freundeskreises im Stiftungsrat, spricht auf Anfrage von einer sehr sachlichen Sitzung: Ziel sei es, sich weiterhin an Budget und Zeitplan zu halten.",Etat
20,"Kralinger: ""Diese Welle erfasst alle Bereiche unseres Wirtschaftslebens"". - ""Politik muss Verantwortung übernehmen"". Wien - Verlegerpräsident Thomas Kralinger warnte am Donnerstagabend beim traditionellen Heurigen des Verbandes Österreichischer Zeitungen (VÖZ) vor der zerstörerischen Kraft des digitalen Tsunamis - wohl in Anlehnung an Nicolas Clasen, den Autor des Lehrbuchs Der digitale Tsunami. Nicht nur Medienmanager und Verlage, sondern auch die Medienpolitik müssen laut Kralinger auf diese Entwicklung reagieren. Diese Welle erfasst alle Bereiche unseres Wirtschaftslebens. Insbesondere im Medienbereich entwickelt sie eine zerstörerische Kraft. Deshalb liegt es an uns Medienmanagern, neue Einnahmequellen außerhalb des traditionellen Geschäfts zu erschließen, um die Zukunft des Journalismus zu sichern.",Etat
28,"Allzu oft hat man als Seher das Gefühl, dass der Regisseur gesagt hat: ""Ganz wunderbar, diese Anekdote! Aber ist sie noch ein bisschen authentischer?"" Trommeln Sie ein paar Berühmtheiten aus Musik, Theater, Radio und Farbfernsehen zusammen, deren Gemeinsamkeit darin besteht, im selben Wiener Bezirk aufgewachsen zu sein. Dann setzen Sie sich an eine Heutheke und nennen Sie sie entspannt, spontan und frei von der Leber, um sie zu unterhalten, ihr Grätzl, ihren Gasserl. Bedenken Sie allerdings, dass es sich um eine ORF-Produktion handelt: Das Ergebnis sind G'schichtln, die sowieso ganz nett und sowieso recht amüsant sind. Wohingegen sie ohnehin ganz im Wiener Ausdruck des 50 Prozent Superlativs Salzburg sind. Also mehr oder weniger halt. Allzu oft hat man als Seher als Seher das Gefühl, der Regisseur habe Halt, Halt, Halt, C",Etat
63,"Am Sonntagabend diskutierte Ingrid Thurnher mit Vertretern der Parteien. Wien - Mit einem massiv verschärften Asylrecht will sich die Regierung gegen den Zustrom von Flüchtlingen wappnen - die Gesetzesänderung ist heftig umstritten und stößt bei Hilfsorganisationen und Verfassungsexperten teilweise auf heftigen Widerstand. Gibt Österreich internationale Standards in Menschenrechtsfragen auf oder ist es ein legitimer Schritt, um öffentliche Ordnung und innere Sicherheit zu gewährleisten - wie die Regierung sagt? Und wie wirkt sich diese Debatte auf die bevorstehende Wahl aus? Ingrid Thurnher diskutierte darüber: Nutzen Sie das Forum, um sich mit anderen über das Programm auszutauschen.",Etat
71,"Kate und Camilla Parker-Bowles würden sich zanken, seit die schwangere Schwiegermutter geraten hat, sich Sorgen zu machen. William würde sich nach der Geburt des Thronfolgers einen Liebhaber für sie suchen. Sommer 2014 verbrachte er ein wenig Zeit an der Seite von Anna Natrebko (als Luna). Gott sei Dank war Placido Domingo jetzt wieder Teil der Salzburger Festspiele. Nicht als Luna. Bei einer Gala stieß er singend unter anderem mit Rolando Villazón auf offenen Buden an. ZiB2 war dabei, stolze 40 Jahre Domingo in Salzburg. Und: Beruhigend, wenn eine Konstante des Operngeschäfts auftaucht. Würde Domingo gehen, wäre es, als würde sich Camilla Parker aus dem Publikum zurückziehen. Der Ve